In [1]:
import open_clip
from PIL import Image
import torch
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'


/home/wht/anaconda3/envs/brain/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
model_type = 'ViT-bigG-14'
pretrain_map={
                'RN50':'openai',
                'RN101':'openai',
                'RN50x4':'openai',
                'RN50x16':'openai',
                'RN50x64':'openai',
                'ViT-B-16':'laion2b_s34b_b88k',
                'ViT-B-32':'laion2b_s34b_b79k',
                'ViT-L-14':'laion2b_s32b_b82k',
                'ViT-H-14':'laion2b_s32b_b79k',
                'ViT-g-14':'laion2b_s34b_b88k', 
                'ViT-bigG-14':'laion2b_s39b_b160k'}
vlmodel, preprocess,_ = open_clip.create_model_and_transforms(model_type, device="cuda:1",pretrained=pretrain_map[model_type])
vlmodel.eval()


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1664, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((1664,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-47): 48 x ResidualAttentionBlock(
          (ln_1): LayerNorm((1664,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1664, out_features=1664, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((1664,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1664, out_features=8192, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=8192, out_features=1664, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((1664,), eps=1e-05, elementwi

In [10]:

image_path = "/home/wht/multimodal_brain/src/tasks/plot/llama_07s.jpg"
image = Image.open(image_path)

image_tensor = preprocess(image).unsqueeze(0).to("cuda:1")
model = vlmodel.visual
# model.output_tokens = True
# model.attn_pool = model.proj = None
# pooler_output, last_hidden_state = model(image_tensor)
# print(pooler_output.shape)
# print(last_hidden_state.shape)
pooler_output= vlmodel.encode_image(image_tensor)
print(image_tensor.shape)
print(pooler_output.shape)

torch.Size([1, 3, 224, 224])
torch.Size([1, 1280])


torch.Size([1, 3, 224, 224])
torch.Size([1, 768])


torch.Size([1, 640])


torch.Size([1, 3, 288, 288])

In [7]:
preprocess

Compose(
    RandomResizedCrop(size=(448, 448), scale=(0.9, 1.0), ratio=(0.75, 1.3333), interpolation=bicubic, antialias=True)
    <function _convert_to_rgb at 0x7fb3f6cbf940>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [8]:
last_hidden_state = torch.cat([pooler_output.unsqueeze(1), last_hidden_state], dim=1)
last_hidden_state.shape

torch.Size([1, 50, 768])

: 

In [2]:
import multiprocessing

# 获取 CPU 核心数
cpu_count = multiprocessing.cpu_count()

print(f"CPU 核心数: {cpu_count}")


CPU 核心数: 144


In [13]:
data_path= '/dev/shm/wht/datasets/things-eeg-small/Preprocessed_data_250Hz_whiten/sub-08/test.pt'
loaded_data = torch.load(data_path)
loaded_data['eeg']=torch.from_numpy(loaded_data['eeg'])
dec_list = []
repeat_time = 80

selected_ch = ['P7', 'P5', 'P3', 'P1','Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8','O1', 'Oz', 'O2']
channels = ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F7', 'F5', 'F3',
                        'F1', 'F2', 'F4', 'F6', 'F8', 'FT9', 'FT7', 'FC5', 'FC3', 'FC1', 
                        'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'FT10', 'T7', 'C5', 'C3', 'C1',
                        'Cz', 'C2', 'C4', 'C6', 'T8', 'TP9', 'TP7', 'CP5', 'CP3', 'CP1', 
                        'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'TP10', 'P7', 'P5', 'P3', 'P1',
                        'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8',
                        'O1', 'Oz', 'O2']

if selected_ch:
    selected_idx = [channels.index(ch) for ch in selected_ch]
    loaded_data['eeg'] = loaded_data['eeg'][:,:,selected_idx]
eeg = loaded_data['eeg'].to(torch.float32)
eeg.shape

/tmp/ipykernel_1037800/2440405009.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_data = torch.load(data_path)


torch.Size([200, 80, 17, 250])

In [18]:
mean_data = eeg.mean(axis=1)
mean_data.shape

torch.Size([200, 17, 250])

In [23]:
import os,sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../')))
from base.DLinear import series_decomp

In [26]:
sd = series_decomp(25)
res, moving_mean = sd(mean_data)
print(res.shape, moving_mean.shape)

torch.Size([200, 17, 250]) torch.Size([200, 17, 250])


: 